## Plotting data using panel

This script plot different types of sensor data and allow the user to interact with the plots using different panel widgets to explore data more easily. 
The plots also allow the user to change several parameters and observe the effect that they have on the plots.

Import required packages:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
import panel as pn
import data_processing_methods as dpm
import datetime as dt

intial plot contaning all the sensor data channels 

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')

plt.rcParams["figure.figsize"] = (20,20)
df_merged.plot(subplots=True, layout=(7,5))
plt.show()

**Display roof temperatures**

All the sensor data collected relating to roof temperature is plotted using holoviews and the low pas filtering parameter can be adjusted to obtain a smooth curve

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
cols = list(df_merged.columns[df_merged.columns.str.startswith('ROOF_0104_300_')])
y = pn.widgets.Select(name='Variable to filter - constant low pass filter', options=cols ,value='ROOF_0104_300_10_TC')

@pn.depends(y.param.value)
def plot_furnace_data(y):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('ROOF_0104_300_')]
    
    fig_a = df_merged_truncated.hvplot(height = 500, width = 1200, line_width = 5)
    df_merged_filtered = df_merged_truncated.copy()
    df_merged_filtered.loc[:,y] = dpm.low_pass_filter(df_merged_truncated.loc[:,y])
    fig_b = df_merged_filtered.hvplot.line(y = y, height = 500, width = 1200, color= 'black',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.Column(pn.pane.Markdown("""
# Calorific Value (CV) - Furnace data
""", width=500),
pn.pane.Markdown("""
# Roof temperatures
""", width=500))

# Create interactive panels with radio buttons and checkboxes
chart_a_interact = pn.interact(plot_furnace_data,y=y)
layout_a = pn.Column(explanation_pane, chart_a_interact)
layout_a.servable()

Further parameter adjustments added to improve the signal output (e.g spike removal)

In [ ]:
import datetime as dt
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

cols = list(df_merged.columns[df_merged.columns.str.startswith('ROOF_0104_300_')])
y = pn.widgets.Select(name='Chose a signal to tune filter parameters', options=cols ,value='ROOF_0104_300_10_TC')

slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date', value=dt.datetime(2022, 10, 9))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 13))

@pn.depends(y.param.value)
def plot_trunc_furnace_data(y,slider_pos, date_start, date_end, Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('ROOF_0104_300_')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_a = df_merged_truncated.hvplot(height = 500, width = 1200, line_width = 5)
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,y] = dpm.remove_spikes(df_merged_truncated.loc[:,y],olr_def=1)
    
    df_merged_filtered.loc[:,y] = dpm.low_pass_filter(df_merged_filtered.loc[:,y],wn=slider_pos)
    fig_b = df_merged_filtered.hvplot.line(y = y, height = 500, width = 1200, color= 'black', label = 'Filtered signal',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
## Processing signals
""", width=500)

# Create interactive panels with radio buttons and checkboxes
chart_b_interact = pn.interact(plot_trunc_furnace_data,y=y,slider_pos=slider,date_start=dt_start_input, date_end=dt_end_input)
layout_b = pn.Column(explanation_pane, chart_b_interact)
layout_b.servable()

**Diaplay PID Zone 1 sensor data**

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
cols = list(df_merged.columns[df_merged.columns.str.startswith('PID_ZONE_1')])
y = pn.widgets.Select(name='Variable to filter - constant low pass filter', options=cols ,value='PID_ZONE_1_PV')

@pn.depends(y.param.value)
def plot_PID_data(y):
    
    df_merged_truncated = df_merged.loc[:, df_merged.columns.str.startswith('PID_ZONE_1')]
    df_merged_filtered = df_merged_truncated.copy()
    
    fig_a = df_merged_truncated.hvplot(height = 500, width = 1200, line_width = 5)
    
    df_merged_filtered.loc[:,y] = dpm.low_pass_filter(df_merged_truncated.loc[:,y])
    fig_b = df_merged_filtered.hvplot.line(y = y, height = 500, width = 1200, color= 'black',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# PID Zone 1
""", width=500)

# Create interactive panels with radio buttons and checkboxes
chart_c_interact = pn.interact(plot_PID_data,y=y)
layout_c = pn.Column(explanation_pane, chart_c_interact)
layout_c.servable()

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

cols = list(df_merged.columns[df_merged.columns.str.startswith('PID_ZONE_1')])
y = pn.widgets.Select(name='Chose a signal to tune filter parameters', options=cols ,value='PID_ZONE_1_PV'

slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date', value=dt.datetime(2022, 9, 14))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 9, 28))

@pn.depends(y.param.value)
def plot_trunc_PID_data(y,slider_pos, date_start, date_end, Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('PID_ZONE_1')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_a = df_merged_truncated.hvplot(height = 500, width = 1200, line_width = 5)
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,y] = dpm.remove_spikes(df_merged_truncated.loc[:,y],olr_def=1)
    
    df_merged_filtered.loc[:,y] = dpm.low_pass_filter(df_merged_filtered.loc[:,y],wn=slider_pos)
    fig_b = df_merged_filtered.hvplot.line(y = y, height = 500, width = 1200, color= 'black', label = 'Filtered signal',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
## Processing signals
""", width=500)

# Create interactive panels with radio buttons and checkboxes
chart_b_interact = pn.interact(plot_trunc_PID_data,y=y,slider_pos=slider,date_start=dt_start_input, date_end=dt_end_input)
layout_b = pn.Column(explanation_pane, chart_b_interact)
layout_b.servable()

**Display air flow**

Air flow is plotted using raw data:

In [ ]:
# Read data realting to air flow
df_merged = pd.read_pickle('merged_sensor_df.pkl')
cols = list(df_merged.columns[df_merged.columns.str.startswith('AIR_')])

df_merged_selected = df_merged.loc[:, df_merged.columns.str.startswith('AIR_')]
df_merged_filtered = df_merged_selected.copy()

for col in cols:
    df_merged_filtered.loc[:,col] = dpm.remove_spikes(df_merged_selected.loc[:,col],olr_def=1)
    df_merged_filtered.loc[:,col] = dpm.low_pass_filter(df_merged_filtered.loc[:,col],wn=0.1)

def plot_air_data():
      
    fig_c = df_merged_filtered.hvplot(height = 500, width = 1200, line_width = 5)
    
    return fig_c

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Air Flow
""", width=500)

# Create interactive panels with radio buttons and checkboxes
layout_c = pn.Column(explanation_pane, plot_air_data)
layout_c.servable()

**Display gas flow**

Gas flow is plotted using raw data:

In [ ]:
# Read data relating to gas flow
df_merged = pd.read_pickle('merged_sensor_df.pkl')
cols = list(df_merged.columns[df_merged.columns.str.startswith('GAS_')])

df_merged_selected = df_merged.loc[:, df_merged.columns.str.startswith('GAS_')]
df_merged_filtered = df_merged_selected.copy()

for col in cols:
    df_merged_filtered.loc[:,col] = dpm.remove_spikes(df_merged_selected.loc[:,col],olr_def=1)
    df_merged_filtered.loc[:,col] = dpm.low_pass_filter(df_merged_filtered.loc[:,col],wn=0.01)

def plot_gas_data():
       
    fig_d = df_merged_selected.hvplot(height = 500, width = 1200, line_width = 5)
    
    return fig_d

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Gas Flow
""", width=500)

# Create interactive panels with radio buttons and checkboxes
layout_c = pn.Column(explanation_pane, plot_gas_data)
layout_c.servable()

**Disply O2 output**

Several data filtering techniques are used to improve the signal quality

In [ ]:
df_merged = pd.read_pickle('merged_sensor_df.pkl')
df_merged = df_merged.resample('1T').mean()

# create widgets
slider = pn.widgets.FloatSlider(start=0.01, end=0.1, step =0.01, value=0.1, name='low pass filter (normalised cut-off freq.)')

dt_start_input = pn.widgets.DatetimeInput(name='Start date (choose a 4 day period for parameter tuning)', value=dt.datetime(2022, 9, 6))
dt_end_input = pn.widgets.DatetimeInput(name='End date', value=dt.datetime(2022, 10, 14))

# Define function for ploting  O2 sensor data
def plot_O2_data(slider_pos, date_start, date_end, Remove_spikes = False):

    df_merged_truncated = df_merged.loc[:,df_merged.columns.str.startswith('FURNACE_0126_341_04_O2')]
    df_merged_truncated = df_merged_truncated [date_start: date_end]
    fig_a = df_merged_truncated.hvplot(ylabel = 'O2 output',height = 500, width = 1600, line_width = 5, grid= True,legend='left')
    df_merged_filtered = df_merged_truncated.copy()
    if Remove_spikes:
        df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'] = dpm.remove_spikes(df_merged_truncated.loc[:,'FURNACE_0126_341_04_O2'],olr_def=1)
    
    df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'] = dpm.low_pass_filter(df_merged_filtered.loc[:,'FURNACE_0126_341_04_O2'],wn=slider_pos)
    fig_b = df_merged_filtered.hvplot.line(y = 'FURNACE_0126_341_04_O2', height = 500, width = 1600, color= 'black', label = 'Filtered signal',line_width = 2)
    
    return fig_a*fig_b

In [ ]:
# Plot 
explanation_pane = pn.pane.Markdown("""
# Output O2
""", width=500)

# Create interactive panels
chart_b_interact = pn.interact(plot_O2_data,date_start=dt_start_input, date_end=dt_end_input,slider_pos=slider)
layout_b = pn.Column(explanation_pane, chart_b_interact)
layout_b.servable()